In [5]:
%load_ext autoreload
%autoreload 2
from data import load_articles, save_articles, load_summaries, save_summaries
from model_wrappers import GPTWrapper, ClaudeWrapper, Llama3Wrapper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
articles = load_articles()
len(articles)

2000

In [3]:
article = articles['35232142']
article

Article(dataset='xsum', id='35232142', text='The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and th

In [4]:
gpt35 = GPTWrapper('gpt35', 'gpt-3.5-turbo')
opus = ClaudeWrapper('claude', 'claude-3-opus-20240229')

In [7]:
llama3_8b = Llama3Wrapper('llama3_8b', 'meta-llama/Meta-Llama-3-8B-Instruct')

Loading checkpoint shards: 100%|██████████| 4/4 [00:13<00:00,  3.36s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
keys = list(articles.keys())
for key in keys:
    print(llama3_8b.summarize(articles[key]))
    print()

Flooding in Newton Stewart and surrounding areas of Scotland's Dumfries and Galloway and Borders regions caused widespread damage, with repair work ongoing and a flood alert remaining in place.

A fire broke out at a Holiday Inn car park in Northern Ireland, destroying two tour buses carrying passengers from Germany and China/Taiwan, and police are investigating the deliberate ignition.



ValueError: Input length of input_ids is 1197, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
type(x)

list